#### Imports

In [1]:
# Redes neurais, funções de ativação, otimizadores e tensores
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
# Dados, Conjuntos e Transformações
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
import torchvision.transforms as transforms
import string # para a construção de um dicionário essencial ao sistema
# io do Sistema Operacional (para ler base de dados)
import os
from os import walk
# Manipulação Gráfica (para plotar imagens e gráficos) e randomizações
from skimage import transform # função para modificar facilmente o tamanho de uma imagem
import matplotlib.pyplot as plt
import numpy as np
import random
from datetime import datetime
import math

###### Setup do Estado de Execução

In [2]:
# semente de random
np.random.seed(random.seed(datetime.now()))

# dispositivo de calculo
dispositivo = 'cuda' if torch.cuda.is_available() else 'cpu'

#### Classe Dataset

In [3]:
class DatasetMassey(Dataset):
    def __init__(self,raiz='Massey/handgestures_combinado/', transform=None):
        #construtor da superclasse
        super(DatasetMassey, self).__init__()
        #ler dados
        self.x = []
        self.y = []
        self.transform = transform
        self.raiz = raiz
        # loops para leitura de dados e preenchimento de X(dado) e Y(label)
        for (_, _, filenames) in walk(raiz):
            self.x.extend(sorted(filenames)) # adiciona o nome de arquivo a X
            break
        # para cada exemplo de X, pegue a label e armazene no dataset
        for exemplo in self.x:
            self.y.extend(exemplo[6])
        
        #armazene o tamanho do dataset
        self.n_amostras = len(self.x)
        self.n_classes = np.unique(self.y)
        
    def __getitem__(self, indice):
        # Só aqui no GetItem, as imagens serão lidas de fato
        # Isso ajuda na eficiência de memória e tempo, pois só lemos e carregamos do disco ao precisarmos delas
        
        caminho_de_arquivo = os.path.join(self.raiz, self.x[indice])
        amostra = {'imagem':plt.imread(caminho_de_arquivo),'label':self.y[indice]}
        
        if self.transform:
            amostra = self.transform(amostra)
        
        return amostra
    
    def __len__(self):
        return self.n_amostras


#### Dataloaders

In [4]:
def CriarDataloaders(dataset=None, tamanho_batch=4, bagunçar=True, porcentagem_split=.2):
    
    if dataset is not None:   
        dataset = dataset
        indices = list(range(len(dataset)))
        split = int(np.floor(porcentagem_split * len(dataset)))

        if bagunçar :            
            # MISTURA OS INDICES
            np.random.shuffle(indices)

        indices_treino, indices_validação = indices[split:], indices[:split]

        sampler_treino = SubsetRandomSampler(indices_treino)
        sampler_validação = SubsetRandomSampler(indices_validação)

        loader_treino = torch.utils.data.DataLoader(dataset, batch_size=tamanho_batch, sampler=sampler_treino)
        loader_validação = torch.utils.data.DataLoader(dataset,batch_size=tamanho_batch, sampler=sampler_validação)

        return loader_treino, loader_validação
    else:
        print("Sem dataset = sem dados, sem dados = sem dataloaders")
        return None,None

#### Transforms

In [5]:
class MinhaRescale(object):
    def __init__(self, output_size):
        assert isinstance(output_size, (int, tuple))
        self.output_size = output_size
   
    def __call__(self, amostra):
        imagem, label = amostra['imagem'], amostra['label']
        h, w = imagem.shape[:2]
        
        new_h, new_w = self.output_size
        new_h, new_w = int(new_h), int(new_w)
        img = transform.resize(imagem, (new_h, new_w))
        label = label
        return {'imagem': img, 'label': label}

In [6]:
class ToTensor(object):
    def __init__(self):
        # Cria uma lista com os caracteres em lowercase
        letras = list(string.ascii_lowercase)
        
        # Cria uma lista com os dígitos numéricos decimais
        numeros = list(string.digits)
        
        # Cria a lista unida de Dígitos e Letras (Que é todo o nosso conjunto de labels do Massey)
        A_Grande_Lista = numeros+letras
        
        # Cria um dicionário vazio que será preenchido, mapeando Label -> índice numérico
        self.O_Grande_Dicionario = dict()
        
#         print(A_Grande_Lista)

        # Preenche o dicionário, mapeando cada item da lista de labels para um índice numérico
        for i in range(len(A_Grande_Lista)):
            self.O_Grande_Dicionario[A_Grande_Lista[i]] = i
            if(i>=len(numeros)):
                self.O_Grande_Dicionario[i] = A_Grande_Lista[i]
        
#         print(self.O_Grande_Dicionario)
        
    def __call__(self, amostra):
        imagem, label = amostra['imagem'], amostra['label']
        # Sobre a Label
            # Devemos trocar a String por seu índice numérico no dicionário
            # Com o índice numérico podemos transformar em um torch.Tensor
        label = self.O_Grande_Dicionario[label]
        label = torch.tensor(label)
        
        # Sobre a Imagem
            # Devemos Trocar os eixos pois:
            # imagem numpy: H x W x Canais
            # imagem torch: Canais X H X W
        imagem = imagem.transpose((2, 0, 1))
        
        # Retorno da Tupla de Tensores
        return {'imagem': torch.from_numpy(imagem),
                'label': label}
    
    # Função que retorna O Grande Dicionario
    def getDicionario(self):
        return self.O_Grande_Dicionario
    
    # Função que converte label de volta para String
        # Só serve pra verificar se tá tudo OK
    def TensorString(self, tensor):
        item = tensor.item()
        if( item > 9):
            return self.O_Grande_Dicionario[item]
        else:
            return str(item)

In [7]:
# Esta só serve para transformar imagens TorchTensor em Imagens do Numpy
class ToNumpyImage(object):
    def __call__(self, tensor_imagem):
        imagem = tensor_imagem.permute(1, 2, 0)
        return imagem

#### Funções de Treino e Validação

In [8]:
def treinar(loader_treino, modelo, n_epocas, fun_custo, otimizador,dispositivo,nome_modelo="não passado",nome_otimizador="não passado",indice_execução=0):
    print("Iniciado o Treinamento Número [ "+str(indice_execução)+ " ] - Modelo:",nome_modelo," Com Otimizador:",nome_otimizador)
    modelo = modelo.to(dispositivo)
    modelo.train()
    
    n_passos = len(loader_treino)
    for epoca in range(n_epocas):
        for passo, amostra in enumerate(loader_treino):
            imagens = amostra['imagem'].to(dispositivo)
            labels = amostra['label'].to(dispositivo)

            saidas = modelo(imagens)
            custo = fun_custo(saidas,labels)

            custo.backward()
            otimizador.step()
            otimizador.zero_grad()
        
        print(f'Época [ {epoca+1} / {n_epocas} ]\t|\tCusto [ {custo} ]')
    print("Finalizado o Treinamento do Modelo")
    return modelo, fun_custo, otimizador

In [9]:
def validar(loader_validação, modelo, dispositivo):
    modelo.to(dispositivo)
    modelo.eval()
    n_corretos = 0
    n_amostras = 0
    with torch.no_grad():
        for exemplos in (loader_validação):
            imagens = exemplos['imagem'].to(dispositivo)
            labels = exemplos['label'].to(dispositivo)

            saidas = modelo(imagens)
            _, predição = torch.max(saidas,1)
            n_amostras += labels.size(0)
            n_corretos += (predição == labels).sum().item()
    precisão = 100.0 * n_corretos / n_amostras
    return precisão

In [10]:
def armazenar_media_desvio(lista_precisões,nome_modelo,nome_otimizador):
    precisão_media = sum(lista_precisões)/len(lista_precisões)
    #somatorio: raiz_quadrada((xi - x_media)²) / N
    desvio_padrao = 0.0
    for precisão in lista_precisões: desvio_padrao += math.sqrt(pow((precisão-precisão_media)/len(lista_precisões),2))
    # Armazena as informações
    informações = "Precisão Média = "+str(precisão_media)+", Desvio Padrão da Precisão = "+str(desvio_padrao)
    guardar_report(report=informações,modelo=nome_modelo,otimizador=nome_otimizador)

#### CNN 1 - Arquitetura Lenet (Com Entrada RGB)

In [11]:
class Lenet_RGB(nn.Module):
    def __init__(self, n_classes):
        super(Lenet_RGB,self).__init__()
        
        # Armazena numero de classes do problema
        self.n_classes = n_classes
        
        # Função de Pool que será usada na Rede -> Kernel_Size = 2 , Stride = 2
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        
        # Primeira camada
        self.conv1 = nn.Conv2d(in_channels= 3, out_channels= 6, kernel_size=5)
            
        # Segunda camada
        self.conv2 = nn.Conv2d(in_channels= 6, out_channels= 16, kernel_size=5)
        
        # Treceira camada
        self.conv3 = nn.Conv2d(in_channels= 16, out_channels= 120 , kernel_size=5)
        
        # Primeira camada FC
        self.fc1 = nn.Linear(in_features= 120, out_features= 84)
        
        # Segunda camada FC
        self.fc2 = nn.Linear(in_features=84, out_features= self.n_classes)
        
    def forward(self,entrada):
        
        saida = self.conv1(entrada)
        saida = F.relu(saida)
        saida = self.pool(saida)
        
        saida = self.conv2(saida)
        saida = F.relu(saida)
        saida = self.pool(saida)
        
        saida = self.conv3(saida)
        saida = F.relu(saida)
        
        saida = saida.view(-1,120)
        
        saida = self.fc1(saida)
        saida = F.relu(saida)
        
        saida = self.fc2(saida)
        
        return saida

#### CNN 2 - Lenet Modificada (Quantidade de Canais de Saída das Camadas Convolucionais Aumentada)

In [12]:
class Lenet_RGB_Mod(nn.Module):
    def __init__(self, n_classes):
        super(Lenet_RGB_Mod,self).__init__()
        
        # Armazena numero de classes do problema
        self.n_classes = n_classes
        
        # Função de Pool que será usada na Rede -> Kernel_Size = 2 , Stride = 2
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        
        # Primeira camada
        self.conv1 = nn.Conv2d(in_channels= 3, out_channels= 18, kernel_size=5)
            
        # Segunda camada
        self.conv2 = nn.Conv2d(in_channels= 18, out_channels= 32, kernel_size=5)
        
        # Treceira camada
        self.conv3 = nn.Conv2d(in_channels= 32, out_channels= 160 , kernel_size=5)
        
        # Primeira camada FC
        self.fc1 = nn.Linear(in_features= 160, out_features= 84)
        
        # Segunda camada FC
        self.fc2 = nn.Linear(in_features=84, out_features= self.n_classes)
        
    def forward(self,entrada):
        
        saida = self.conv1(entrada)
        saida = F.relu(saida)
        saida = self.pool(saida)
        
        saida = self.conv2(saida)
        saida = F.relu(saida)
        saida = self.pool(saida)
        
        saida = self.conv3(saida)
        saida = F.relu(saida)
        
        saida = saida.view(-1,160)
        
        saida = self.fc1(saida)
        saida = F.relu(saida)
        
        saida = self.fc2(saida)
        
        return saida

#### CNN 3 - MiniVgg

In [13]:
class MiniVgg(nn.Module):
    def __init__(self, n_classes): 
        super(MiniVgg, self).__init__()
        
        # Maxpool usada
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        
        # Camadas convolucionais com normalização de batch
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=64, kernel_size=3, padding = 1)
        self.norm1 = nn.BatchNorm2d(64)
        
        self.conv2 = nn.Conv2d(64, 64, 3, padding = 1)
        self.norm2 = nn.BatchNorm2d(64)
               
        self.conv3 = nn.Conv2d(64, 128, 3, padding = 1)
        self.norm3 = nn.BatchNorm2d(128)
        
        self.conv4 = nn.Conv2d(128, 128, 3, padding = 1)
        self.norm4 = nn.BatchNorm2d(128)
        
        self.conv5 = nn.Conv2d(128, 256, 3, padding = 1)
        self.norm5 = nn.BatchNorm2d(256)
        
        self.conv6 = nn.Conv2d(256, 256, 3, padding = 1)
        self.norm6 = nn.BatchNorm2d(256)

        self.conv7 = nn.Conv2d(256, 512, 3, padding = 1)
        self.norm7 = nn.BatchNorm2d(512)
        
        self.conv8 = nn.Conv2d(512, 512, 3, padding = 1)
        self.norm8 = nn.BatchNorm2d(512)
        
        # Camada totalmente conectada com normalização de batch
        self.fc1 = nn.Linear(512 * 4 * 4, 128)
        self.norm9 = nn.BatchNorm1d(128)
       
        self.fc2 = nn.Linear(128, 64)
        self.norm10 = nn.BatchNorm1d(64)
        
        self.fc3 = nn.Linear(64, n_classes)
        

    def forward(self, x):       
        
        saida = F.elu(self.norm1(self.conv1(x)))
        saida = F.elu(self.norm2(self.conv2(saida)))
        saida = self.pool(saida)
        
        saida = F.elu(self.norm3(self.conv3(saida)))
        saida = F.elu(self.norm4(self.conv4(saida)))
        saida = self.pool(saida)
        
        saida = F.elu(self.norm5(self.conv5(saida)))
        saida = F.elu(self.norm6(self.conv6(saida)))
        saida = self.pool(saida)
        
        saida = F.elu(self.norm7(self.conv7(saida)))
        saida = F.elu(self.norm8(self.conv8(saida)))
        
        saida = saida.view(-1, 512 * 4 * 4)
        
        saida = F.elu(self.norm9(self.fc1(saida)))
        saida = F.elu(self.norm10(self.fc2(saida)))
        saida = self.fc3(saida)

        return saida

###### Classe Para Reduzir Código (retorna novas instancias de otimizadores e taxas de aprendizado diferentes)

In [14]:
class Getter_Parametro():
    def __init__(self):
        self.nomes_fun_custo = ['Entropia Cruzada','Erro Quadrático Médio','Logaritmo Negativo']
        self.funções_de_custo = {'Entropia Cruzada':nn.CrossEntropyLoss,'Erro Quadrático Médio':nn.MSELoss,'Logaritmo Negativo':nn.BCELoss}
        
        self.nomes_otimizador = ['Gradiente Descendente','Adagrad','Adam']
        self.otimizadores = {'Gradiente Descendente':torch.optim.SGD,'Adagrad':torch.optim.Adagrad,'Adam':torch.optim.Adam}
        
        self.taxas_aprendizado = [0.5,0.1,0.05,0.001]
        self.indice_taxa =0

    def get_otimizador(self,indice=0,quero_o_nome=False):
        if quero_o_nome:
            return self.nomes_otimizador[indice]
        else:
            return self.otimizadores[self.nomes_otimizador[indice]]
        
    def get_fun_custo(self,indice=0, quero_o_nome=False):
        if quero_o_nome:
            return self.nomes_fun_custo[indice]
        else:
            return self.funções_de_custo[self.nomes_fun_custo[indice]]()
    
    def get_lr(self):
        taxa = self.taxas_aprendizado[self.indice_taxa]
        self.indice_taxa += 1
        self.indice_taxa %= len(self.taxas_aprendizado)
        return taxa
    

In [15]:
# Função que armazena mais uma linha no arquivo de report
def guardar_report(report,modelo,otimizador):
    arquivo = open('resultados/report_Modelo-{}_Otimizador-{}'.format(modelo,otimizador),'a+')
    arquivo.write('{}{}'.format(report,'\n'))
    arquivo.close()

---
---
# Execução
---

###### Insira Aqui o Número de Execuções e Épocas de Treinaento para cada treino Desejadas 

In [16]:
# Definindo a Transformação a ser utilizada
composta = transforms.Compose([MinhaRescale((32,32)),ToTensor()])

# Instanciando Novo DataSet
ds_massey = DatasetMassey(raiz="Massey/handgestures_combinado/",transform=composta)

In [17]:
N = 20 #30
épocas = 10 #10

In [18]:
gp = Getter_Parametro()
modelos = {'Lenet_RGB':Lenet_RGB,'Lenet_RGB_MODIFICADA':Lenet_RGB_Mod,'MiniVgg':MiniVgg}

# Para cada modelo na lista de classes de modelo, execute N vezes o treino e validação variando: Taxas de Aprendizado e Otimizadores
for taxa_aprendizado in [0.5,0.1,0.05,0.001]:
    for modelo in modelos.keys(): # Para cada modelo
        for indice_otimizador in range(3): # Para cada otimizador
            # Cria lista vazia para armazena a precisão de cada rede ao ser treinada na época Ni
            precisões = [0 for i in range(N)]

            for execução in range(N): # Para cada execução
                report=""
                # Instancie um novo loader de treino e validação a cada execução, aleatorizando novamente os índices
                loader_treino, loader_validação = CriarDataloaders(dataset=ds_massey,bagunçar=True,porcentagem_split=.2)

                # Instancie um modelo da lista de modelos para ser treinado e validado
                modelo_instanciado = modelos[modelo](36).to(dispositivo)

                # Instancie nova função de custo (Entropia Cruzada) -> (guarda historico)
                fun_custo = gp.get_fun_custo()

                # Escolha uma nova taxa de aprendizagem (da lista cíclica de taxas)
                    # Ciclo de taxas de aprendizado [ -> 0.5 -> 0.1 -> 0.05 -> 0.001] muda para o prox a cada execução
                report+="Taxa de Aprendizado = "+str(taxa_aprendizado)+', '

                # Instancie um otimizador
                    # Ciclo de otimizadores [ -> 'Gradiente Descendente' -> 'Adagrad' -> 'Adam' ]
                otimizador = gp.get_otimizador(indice_otimizador)(modelo_instanciado.parameters(),lr=taxa_aprendizado)
                report += "Otimizador = "+gp.get_otimizador(indice_otimizador,True)+', Número de Épocas = '+str(épocas)+', '

                # Treine o modelo instanciado com os seus parâmetros (n epocas, otimizador, loader de treino)
                modelo_instanciado,_,_ = treinar(loader_treino,modelo_instanciado,épocas,fun_custo,otimizador,dispositivo,modelo,gp.get_otimizador(indice_otimizador,True),execução)

                ####### CÓDIGO PARA VALIDAÇÃO #######
                precisão = validar(loader_validação,modelo_instanciado,dispositivo)
                precisões[execução] = precisão
                report += "Precisão = "+"{:.2f}".format(precisão)+"%"
                # Armazena linha de report para modelo com otimizador
                guardar_report(report=report,modelo=modelo,otimizador=gp.get_otimizador(indice_otimizador,True))

            # Após as N execuções (preencheu a lista de médias para esta rede com este otimizador), armazene a precisão média e o desvio padrão
            armazenar_media_desvio(lista_precisões=precisões,nome_modelo=modelo,nome_otimizador=gp.get_otimizador(indice_otimizador,True))

Iniciado o Treinamento Número [ 0 ] - Modelo: Lenet_RGB  Com Otimizador: Gradiente Descendente
Época [ 1 / 10 ]	|	Custo [ 3.8008155822753906 ]
Época [ 2 / 10 ]	|	Custo [ 3.462131977081299 ]
Época [ 3 / 10 ]	|	Custo [ 3.539107322692871 ]
Época [ 4 / 10 ]	|	Custo [ 3.517744779586792 ]
Época [ 5 / 10 ]	|	Custo [ 3.5825397968292236 ]
Época [ 6 / 10 ]	|	Custo [ 3.4797096252441406 ]
Época [ 7 / 10 ]	|	Custo [ 3.6952545642852783 ]
Época [ 8 / 10 ]	|	Custo [ 3.6278669834136963 ]
Época [ 9 / 10 ]	|	Custo [ 3.5419371128082275 ]
Época [ 10 / 10 ]	|	Custo [ 3.6056783199310303 ]
Finalizado o Treinamento do Modelo
Iniciado o Treinamento Número [ 1 ] - Modelo: Lenet_RGB  Com Otimizador: Gradiente Descendente
Época [ 1 / 10 ]	|	Custo [ 3.8238067626953125 ]
Época [ 2 / 10 ]	|	Custo [ 3.820324182510376 ]
Época [ 3 / 10 ]	|	Custo [ 3.628328323364258 ]
Época [ 4 / 10 ]	|	Custo [ 3.6903417110443115 ]
Época [ 5 / 10 ]	|	Custo [ 3.563079357147217 ]
Época [ 6 / 10 ]	|	Custo [ 3.505427837371826 ]
Época [ 7 / 1

Época [ 6 / 10 ]	|	Custo [ 3.603614330291748 ]
Época [ 7 / 10 ]	|	Custo [ 3.649087429046631 ]
Época [ 8 / 10 ]	|	Custo [ 3.758765459060669 ]
Época [ 9 / 10 ]	|	Custo [ 3.462859630584717 ]
Época [ 10 / 10 ]	|	Custo [ 3.7035813331604004 ]
Finalizado o Treinamento do Modelo
Iniciado o Treinamento Número [ 14 ] - Modelo: Lenet_RGB  Com Otimizador: Gradiente Descendente
Época [ 1 / 10 ]	|	Custo [ 3.716265916824341 ]
Época [ 2 / 10 ]	|	Custo [ 3.495204448699951 ]


KeyboardInterrupt: 

- Resumo
    - Cada Modelo tem **20 execuções** para cada tipo de otimizador
    - Cada otimizador executa **N/4 vezes para cada taxa de aprendizagem**
    - O Loader de dados de *Treino e Validação* tem os índices **reatribuídos e aleatorizados a cada execução** (mantendo 20% dos índices para validação)
    - São efetuadas um total de (**<i>Número de Modelos</i> \* N \* *Número de Otimizadores* \* Número de Taxas de Aprendizado**) Treinamentos de **X** épocas]